In [10]:
import glob
import pickle
from ipywidgets import interact
import pipeline
reload(pipeline)
from pipeline import *

def loadAllSamples():
    all_filenames = glob.glob('./data/*.ciSamples.pickle')
    for filename in all_filenames:
        with open(filename) as f:
            for ciSample in pickle.load(f):
                yield ciSample

ciSamples = list(loadAllSamples())

In [11]:
len(ciSamples)

510

In [12]:
import numpy as np
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt


def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

import math

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])


In [36]:
import matplotlib.pyplot as plt
import mpld3

def plotNorms(ciSample, startT=None):
    if startT is not None:
        readingTimes = ciSample.npReadings[:,-1] - ciSample.npReadings[0,-1]
        readings = ciSample.npReadings[(readingTimes > startT) & (readingTimes < startT + 64./20.)]
    else:
        readings = ciSample.npReadings
        
    label = 'type={} "{}"'.format(ciSample.reportedActivityType, ciSample.sampleNotes)
    
    
    if not len(readings):
        print readingTimes
        return
    
    
    times = readings[:,-1] - readings[0,-1]
    norms = np.linalg.norm(readings[:,:-1], axis=1)
    
    plt.figure(figsize=(14, 16))
    
    plt.subplot(2, 1, 1)
    plt.title(label)
    plt.grid(b='on', axis='y')
    plt.xlim(0, 4)
    plt.ylim(0, 3)
    plt.plot(times, norms)
    
    plt.subplot(2, 1, 2)
    plt.grid(b='on', axis='y')
    #plt.xlim(0, )
    #plt.ylim(0, 10)
    
    pass_hz = 2
    
    fx = butter_lowpass_filter(readings[:,0], pass_hz, 20, order=6)
    fy = butter_lowpass_filter(readings[:,1], pass_hz, 20, order=6)
    fz = butter_lowpass_filter(readings[:,2], pass_hz, 20, order=6)
    
#     plt.plot(times, readings[:,0] - fx, label='filtered x')
#     plt.plot(times, readings[:,0] - fy, label='filtered y')
#     plt.plot(times, readings[:,0] - fz, label='filtered z')

#     plt.plot(times, np.arctan2(readings[:,1], readings[:,0]), label='theta = atan(y/x)')
#     plt.plot(times, np.arccos(readings[:,2] / norms), label='phi = acos(z/r)')
#     plt.plot(times, np.arctan2(fy, fx), label='theta')
#     plt.plot(times, np.arccos(fz / np.sqrt(fx*fx + fy*fy + fz*fz)), label='phi')
#     plt.plot(times, np.sqrt(fx*fx + fy*fy + fz*fz), label='norm')


    mean_acc = np.mean(readings[:,:-1], axis=0)
    z_vec = [0, 0, 1]
    
    rotation_axis = np.cross(mean_acc, z_vec)
    theta = np.arccos(np.dot(mean_acc, z_vec) / (np.linalg.norm(mean_acc)))
    mat = rotation_matrix(rotation_axis, theta)
    rotated = np.array([np.dot(mat, vec) for vec in readings[:,:-1]])
    
    plt.plot(times, rotated[:,0], label='x')
    plt.plot(times, rotated[:,1], label='y')
    plt.plot(times, rotated[:,2], label='z')
    
#     plt.plot(times, rotated[:,2] - norms, label='zdiff')
    
    planar_norms = np.sqrt(rotated[:,0]*rotated[:,0] + rotated[:,1]*rotated[:,1])
#     freqs = np.abs(np.fft.rfft(planar_norms * np.hamming(len(planar_norms))))
#     plt.plot(freqs[1:])
    
#     plt.plot(times, np.sqrt(rotated[:,0]*rotated[:,0] + rotated[:,1]*rotated[:,1]), label='horiz')

#     plt.plot(times, rotated[:,0] - rotated[:,1], label='h diff')
    
#     plt.plot(times, np.linalg.norm(rotated, axis=1), label='norm')
    
    plt.xlim(0, 4)
    plt.ylim(-2, 2)
    
    plt.tight_layout()


    plt.legend()
    
    return mpld3.display()

In [37]:
foot = [ciSample for ciSample in ciSamples if ciSample.reportedActivityType in (4,1)]
walking = [ciSample for ciSample in ciSamples if ciSample.reportedActivityType in (4,)]
biking = [ciSample for ciSample in ciSamples if ciSample.reportedActivityType in (2,)]

In [38]:
np.mean(walking[24].npReadings[:,:-1], axis=0)

array([ 0.16528189,  0.9827659 ,  0.08509395])

In [39]:
interact(lambda startT: plotNorms(walking[24], startT), startT=[0, 28])

<function __main__.<lambda>>

In [40]:
interact(lambda startT: plotNorms(biking[71], startT), startT=(0, 28))

<function __main__.<lambda>>

In [213]:
interact(lambda index, startT: plotNorms(walking[index], startT=startT), index=[0, len(walking)-1], startT=(0, 20))

<function __main__.<lambda>>

In [207]:
interact(lambda index, startT: plotNorms(biking[index], startT=startT), index=[0, len(biking)-1], startT=(0, 20))

In [135]:
interact(lambda index, startT: plotNorms(foot[index], startT=startT), index=[0, len(foot)-1], startT=(0, 20))